In [179]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from requests import get, Session, post
import time
from datetime import datetime, timedelta
from io import StringIO
import statistics as stats
import scipy.stats as scistats

In [180]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"



siteCred = {'identity': "space.state@aol.com", 'password': "123SSptaactee123!"}


# use requests package to drive the RESTful session with space-track.org
with Session() as session:
    # run the session in a with block to force session to close if we exit

    # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    print(resp.headers)

    seshcook = resp.headers['Set-Cookie']
    seshcook = seshcook.split(' ')[0][:-1]

{'Date': 'Fri, 22 Nov 2024 14:42:42 GMT', 'Server': 'Apache/2.4.52 (Ubuntu)', 'Set-Cookie': 'chocolatechip=tth6lp2s36f0dqb70ruu60v5qaj7nuq8; expires=Fri, 22-Nov-2024 16:42:42 GMT; Max-Age=7200; path=/; HttpOnly', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Status': '200', 'Content-Length': '2', 'Content-Type': 'application/json', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}


In [181]:
spacetrackheaders = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"accept-encoding": "gzip, deflate, br, zstd",
"accept-language": "en-US,en;q=0.9,es;q=0.8",
"cache-control": "max-age=0",
"cookie": f"spacetrack_csrf_cookie=pc9o9c6722kl32quis48b594gl2cmp5c; {seshcook}",
"priority": "u=0, i",
"sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
"sec-ch-ua-mobile": "?1",
"sec-ch-ua-platform": '"Android"',
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36"}

In [182]:
hdrs2 = {"accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
'Authorization': 'TOK:space.state@aol.com',
'accept-encoding':'gzip, deflate, br, zstd',
'accept-language':'en-US,en;q=0.9,es;q=0.8',
'cache-control':'max-age=0',
'cookie':f'spacetrack_csrf_cookie=8u4fvh6rr0rr9bm68ovuf6h2l2v728r2; {seshcook}',
'priority':'u=0, i',
'referer':'https://www.space-track.org/auth/login',
'sec-ch-ua':'"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
'sec-ch-ua-mobile':'?1',
'sec-ch-ua-platform':'"Android"',
'sec-fetch-dest':'document',
'sec-fetch-mode':'navigate',
'sec-fetch-site':'same-origin',
'sec-fetch-user':'?1',
'upgrade-insecure-requests':'1',
'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36"'}

In [183]:
sat_num=27718

In [184]:
today = datetime.today()
today=str(today)
today=today[:10]
today

'2024-11-22'

In [185]:
zr = get(f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/TLE_LINE1%20ASC/EPOCH/2000-01-01--{today}/format/tle",headers=hdrs2)

In [186]:
print(zr.content)

b'1 27718U 03014A   03101.75166644 -.00000663  00000-0  00000+0 0  9992\r\n2 27718 027.1531 031.4014 7820453 179.3625 183.4290 01.64081412    05\r\n1 27718U 03014A   03102.35906410 +.00026897 +00000-0 +52971-2 0  9995\r\n2 27718 027.2439 031.1026 7828366 179.6776 182.4357 01.64306118000011\r\n1 27718U 03014A   03102.96439490  .00151956  15645-5  18255-1 0  9998\r\n2 27718 027.2624 030.9447 7832051 179.8562 180.6844 01.64519867    21\r\n1 27718U 03014A   03102.96478044  .00150354  15680-5  16392-1 0  9996\r\n2 27718 027.2576 030.9235 7833076 179.8251 180.9182 01.64521751    28\r\n1 27718U 03014A   03103.57024869  .00363946  19433-5  23491-1 0  9997\r\n2 27718 027.3020 030.8897 7835517 180.0731 179.7729 01.64820730    38\r\n1 27718U 03014A   03103.57029120  .00346246  19425-5  19960-1 0  9993\r\n2 27718 027.3020 030.8861 7835907 180.0813 179.7278 01.64891859    32\r\n1 27718U 03014A   03103.57030457 +.00081917 +19468-5 +56131-2 0  9998\r\n2 27718 027.2697 030.8695 7835051 180.0600 179.89

In [187]:
oneline = zr.text

In [188]:
tlesraw = oneline.split('\r\n1')

In [189]:
tlesraw

['1 27718U 03014A   03101.75166644 -.00000663  00000-0  00000+0 0  9992\r\n2 27718 027.1531 031.4014 7820453 179.3625 183.4290 01.64081412    05',
 ' 27718U 03014A   03102.35906410 +.00026897 +00000-0 +52971-2 0  9995\r\n2 27718 027.2439 031.1026 7828366 179.6776 182.4357 01.64306118000011',
 ' 27718U 03014A   03102.96439490  .00151956  15645-5  18255-1 0  9998\r\n2 27718 027.2624 030.9447 7832051 179.8562 180.6844 01.64519867    21',
 ' 27718U 03014A   03102.96478044  .00150354  15680-5  16392-1 0  9996\r\n2 27718 027.2576 030.9235 7833076 179.8251 180.9182 01.64521751    28',
 ' 27718U 03014A   03103.57024869  .00363946  19433-5  23491-1 0  9997\r\n2 27718 027.3020 030.8897 7835517 180.0731 179.7729 01.64820730    38',
 ' 27718U 03014A   03103.57029120  .00346246  19425-5  19960-1 0  9993\r\n2 27718 027.3020 030.8861 7835907 180.0813 179.7278 01.64891859    32',
 ' 27718U 03014A   03103.57030457 +.00081917 +19468-5 +56131-2 0  9998\r\n2 27718 027.2697 030.8695 7835051 180.0600 179.89

In [190]:
for i in range(len(tlesraw)):
   tlesraw[i] = tlesraw[i].replace('\r\n','|')

In [191]:
for i in range(len(tlesraw)):
    tlesraw[i] = tlesraw[i].replace(' ','|')
    tlesraw[i] = tlesraw[i].split('|')

In [192]:
tle_df = pd.DataFrame(tlesraw)

tle_df['Output']=''

In [193]:
tle_df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,Output
0,1,27718U,03014A,,,03101.75166644,-.00000663,,00000-0,,...,,,,05,None,None,None,None,None,
1,,27718U,03014A,,,03102.35906410,+.00026897,+00000-0,+52971-2,0,...,None,None,None,None,None,None,None,None,None,
2,,27718U,03014A,,,03102.96439490,,.00151956,,15645-5,...,01.64519867,,,,21,None,None,None,None,
3,,27718U,03014A,,,03102.96478044,,.00150354,,15680-5,...,01.64521751,,,,28,None,None,None,None,
4,,27718U,03014A,,,03103.57024869,,.00363946,,19433-5,...,01.64820730,,,,38,None,None,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11886,,27718U,03014A,,,24325.59910694,-.00000220,,00000-0,,...,162.7681,177.6134,,1.00272359,39541,None,None,None,None,
11887,,27718U,03014A,,,24325.75335604,-.00000220,,00000-0,,...,162.7577,233.3075,,1.00272293,39550,None,None,None,None,
11888,,27718U,03014A,,,24326.61442861,-.00000218,,00000-0,,...,163.7963,183.1154,,1.00272121,39555,None,None,None,None,
11889,,27718U,03014A,,,24326.67052815,-.00000218,,00000-0,,...,163.3401,203.8235,,1.00272166,79187,None,None,None,None,


In [194]:
def get_daytime(index_num):
    jday = tle_df.iloc[index_num][5]

    jday = float(jday)

    date, time = divmod(jday, 1.0)

    date = str(int(date))
    #
    #
    #Next convert the fractional day into seconds:
    #

    timedelta(days=time)
    #and add the two together:
    #
    output = datetime.strptime(date, '%y%j') + timedelta(time)

    return output.strftime("TLE,%b %d %Y %H:%M:%S.%f,1 ")

In [195]:
for i in range(len(tle_df[0])):
    try:
        tle_df[0].iloc[i] = get_daytime(i)
    except:
        pass #impute deletion flag instead of pass, then delete from df later

In [198]:
tle_df[0].iloc[4] = 'poop'

tle_df = tle_df[tle_df[0] != 'poop']

tle_df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,Output
0,"TLE,Jan 01 2031 18:02:23.980416,1",27718U,03014A,,,03101.75166644,-.00000663,,00000-0,,...,,,,05,None,None,None,None,None,
1,"TLE,Jan 02 2031 08:37:03.138240,1",27718U,03014A,,,03102.35906410,+.00026897,+00000-0,+52971-2,0,...,None,None,None,None,None,None,None,None,None,
2,"TLE,Jan 02 2031 23:08:43.719360,1",27718U,03014A,,,03102.96439490,,.00151956,,15645-5,...,01.64519867,,,,21,None,None,None,None,
3,"TLE,Jan 02 2031 23:09:17.030016,1",27718U,03014A,,,03102.96478044,,.00150354,,15680-5,...,01.64521751,,,,28,None,None,None,None,
5,"TLE,Jan 03 2031 13:41:13.159680,1",27718U,03014A,,,03103.57029120,,.00346246,,19425-5,...,01.64891859,,,,32,None,None,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11886,"TLE,Nov 20 2024 14:22:42.839616,1",27718U,03014A,,,24325.59910694,-.00000220,,00000-0,,...,162.7681,177.6134,,1.00272359,39541,None,None,None,None,
11887,"TLE,Nov 20 2024 18:04:49.961856,1",27718U,03014A,,,24325.75335604,-.00000220,,00000-0,,...,162.7577,233.3075,,1.00272293,39550,None,None,None,None,
11888,"TLE,Nov 21 2024 14:44:46.631904,1",27718U,03014A,,,24326.61442861,-.00000218,,00000-0,,...,163.7963,183.1154,,1.00272121,39555,None,None,None,None,
11889,"TLE,Nov 21 2024 16:05:33.632160,1",27718U,03014A,,,24326.67052815,-.00000218,,00000-0,,...,163.3401,203.8235,,1.00272166,79187,None,None,None,None,


In [135]:
type(tle_df[27].iloc[0])

NoneType

In [136]:
tle_df = tle_df.map(lambda x: str(x) if type(x) != type(str) else x)
tle_df = tle_df.map(lambda x: '' if x == 'None' else x)

In [137]:
tle_df = tle_df.map(lambda x: x + ' ' if type(x) == str and len(x)>1 else x)

In [138]:
#tle_df = tle_df.map(lambda x: NaN if x == 'None ' else x)

In [139]:

tle_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Output
0,"TLE,Sep 22 2015 19:16:38.176896,1",40912U,15049P,,,15265.80321964,,.00003135,,00000-0,...,236.8989,248.4132,15.13635155,,,327,,,,
1,"TLE,Sep 22 2015 20:50:22.155936,1",40912U,15049P,,,15265.86831199,,.00003395,,00000-0,...,236.6722,243.1106,15.13636345,,,331,,,,
2,"TLE,Sep 23 2015 04:31:21.923616,1",40912U,15049P,,,15266.18844819,,.00003852,,00000-0,...,234.8700,188.2658,15.13640273,,,383,,,,
3,"TLE,Sep 23 2015 12:54:24.354432,1",40912U,15049P,,,15266.53778188,,.00003729,,00000-0,...,234.0405,291.4424,15.13642511,,,437,,,,
4,"TLE,Sep 23 2015 20:30:06.235776,1",40912U,15049P,,,15266.85423884,,.00003636,,00000-0,...,233.4332,215.3635,15.13644447,,,480,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10974,"TLE,Jan 20 2024 01:00:21.070656,1",40912U,15049P,,,24020.04191054,,.03137908,,24393-5,...,0002358,238.5952,121.5107,16.20881175463442,,,,,,
10975,"TLE,Jan 21 2024 03:35:37.480704,1",40912U,15049P,,,24021.14973936,,.07252078,,25248-5,...,0005230,242.4971,117.5794,16.32832684463621,,,,,,
10976,"TLE,Jan 21 2024 03:35:37.480704,1",40912U,15049P,,,24021.14973936,,.07252078,,25248-5,...,0005230,242.4971,117.5794,16.32832684463621,,,,,,
10977,"TLE,Jan 21 2024 13:51:09.237600,1",40912U,15049P,,,24021.57719025,,.14527648,,26197-5,...,0008751,245.2391,114.8002,16.45639284463696,,,,,,


In [140]:
type(tle_df[27].iloc[0])

str

In [141]:
tle_df.iloc[0][5]

'15265.80321964 '

In [142]:
jday = tle_df.iloc[0][5]

jday = float(jday)

date, time = divmod(jday, 1.0)

date = str(int(date))
#
#
#Next convert the fractional day into seconds:
#

timedelta(days=time)
#and add the two together:
#
output = datetime.strptime(date, '%y%j') + timedelta(time)

output.strftime("%b %d %Y %H:%M:%S.%f,1")

'Sep 22 2015 19:16:38.176896,1'

In [143]:
for i in tle_df.iloc[10000]:
    print(i)
    print(type(i))

TLE,May 16 2023 19:52:06.282336,1  
<class 'str'>
40912U 
<class 'str'>
15049P 
<class 'str'>

<class 'str'>

<class 'str'>
23136.82785049 
<class 'str'>

<class 'str'>
.00040678 
<class 'str'>

<class 'str'>
00000-0 
<class 'str'>

<class 'str'>
96027-3 
<class 'str'>
0
<class 'str'>

<class 'str'>
9995 
<class 'str'>
2
<class 'str'>
40912 
<class 'str'>

<class 'str'>
97.4194 
<class 'str'>
154.0412 
<class 'str'>
0010036 
<class 'str'>
179.1263 
<class 'str'>
259.1044 
<class 'str'>
15.41707538424778 
<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>


In [144]:
#tle_df = tle_df.map(lambda x: [] if type(x) == type(None) else x)

In [145]:
for h in range(28):
    for col in tle_df.columns[:-2]:
        for i in range(len(tle_df[col])):
            if tle_df[col].iloc[i] == '' and tle_df[col].iloc[i] != tle_df[col+1].iloc[i]:
                tle_df[col].iloc[i] = tle_df[col+1].iloc[i] 
                tle_df[col+1].iloc[i] = ''


In [146]:
for col in tle_df.columns[:-1]:
    if tle_df[col].iloc[0] == tle_df[col].iloc[len(tle_df)-1] and tle_df[col].iloc[0] == '':
        tle_df = tle_df.drop(columns=col)



In [147]:
tle_df[3] = '  ' + tle_df[3]
tle_df[8] = ''
tle_df[9] = '   1,2'
tle_df[10] = ' ' + tle_df[10] + '  '
tle_df[14] = ' ' + tle_df[14] + ' '


In [148]:
tle_df['Output']=tle_df.sum(axis=1,skipna=True)

In [149]:
tle_df['Output'] = tle_df['Output'] + '\n'

In [150]:
print(tle_df['Output'].iloc[0])

TLE,Sep 22 2015 19:16:38.176896,1  40912U 15049P   15265.80321964 .00003135 00000-0 17762-3 0   1,2 40912   97.4602 273.9134 0015944  236.8989  248.4132 15.13635155 327 



In [151]:
tle_df['JDATE']=None
for i in range(len(tle_df)):
    tle_df['JDATE'].iloc[i] = tle_df[3].iloc[i]
    tle_df['JDATE'].iloc[i] = tle_df['JDATE'].iloc[i][2:7]

In [152]:
tle_df = tle_df.sort_values('JDATE',ascending=False)
tle_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Output,JDATE
10978,"TLE,Jan 21 2024 13:51:09.237600,1",40912U,15049P,24021.57719025,.14527648,26197-5,35116-3,0,,"1,2",40912,97.3681,54.2491,0008751,245.2391,114.8002,16.45639284463696,,"TLE,Jan 21 2024 13:51:09.237600,1 40912U 1504...",24021
10977,"TLE,Jan 21 2024 13:51:09.237600,1",40912U,15049P,24021.57719025,.14527648,26197-5,35116-3,0,,"1,2",40912,97.3681,54.2491,0008751,245.2391,114.8002,16.45639284463696,,"TLE,Jan 21 2024 13:51:09.237600,1 40912U 1504...",24021
10976,"TLE,Jan 21 2024 03:35:37.480704,1",40912U,15049P,24021.14973936,.07252078,25248-5,12729-2,0,,"1,2",40912,97.3710,53.7482,0005230,242.4971,117.5794,16.32832684463621,,"TLE,Jan 21 2024 03:35:37.480704,1 40912U 1504...",24021
10975,"TLE,Jan 21 2024 03:35:37.480704,1",40912U,15049P,24021.14973936,.07252078,25248-5,12729-2,0,,"1,2",40912,97.3710,53.7482,0005230,242.4971,117.5794,16.32832684463621,,"TLE,Jan 21 2024 03:35:37.480704,1 40912U 1504...",24021
10974,"TLE,Jan 20 2024 01:00:21.070656,1",40912U,15049P,24020.04191054,.03137908,24393-5,15637-2,0,,"1,2",40912,97.3786,52.4806,0002358,238.5952,121.5107,16.20881175463442,,"TLE,Jan 20 2024 01:00:21.070656,1 40912U 1504...",24020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,"TLE,Sep 23 2015 04:31:21.923616,1",40912U,15049P,15266.18844819,.00003852,00000-0,21734-3,0,,"1,2",40912,97.4602,274.2911,0016018,234.8700,188.2658,15.13640273,383,"TLE,Sep 23 2015 04:31:21.923616,1 40912U 1504...",15266
5,"TLE,Sep 23 2015 23:39:24.022080,1",40912U,15049P,15266.98569470,.00003716,00000-0,20979-3,0,,"1,2",40912,97.4598,275.0717,0016109,232.7793,211.8842,15.13645811,505,"TLE,Sep 23 2015 23:39:24.022080,1 40912U 1504...",15266
3,"TLE,Sep 23 2015 12:54:24.354432,1",40912U,15049P,15266.53778188,.00003729,00000-0,21053-3,0,,"1,2",40912,97.4597,274.6328,0016018,234.0405,291.4424,15.13642511,437,"TLE,Sep 23 2015 12:54:24.354432,1 40912U 1504...",15266
1,"TLE,Sep 22 2015 20:50:22.155936,1",40912U,15049P,15265.86831199,.00003395,00000-0,19205-3,0,,"1,2",40912,97.4603,273.9772,0015940,236.6722,243.1106,15.13636345,331,"TLE,Sep 22 2015 20:50:22.155936,1 40912U 1504...",15265


In [113]:
pwd

'c:\\Users\\Green\\Desktop\\coding projects\\celestrack file\\spoofer'

In [114]:
import os


for i in range(len(tle_df)):
    jate = tle_df['JDATE'].iloc[i]
    filepath = f'DATABASE/StateProcessing/StateDatabase/{jate}_Database_States.txt'

    if os.path.exists(filepath):
        with open(filepath, 'a') as file:
            file.write(tle_df['Output'].iloc[i])
            file.close
    else:
        with open(filepath, 'w') as file:
            file.write(tle_df['Output'].iloc[i])
            file.close
    
    

In [115]:
tle_df.iloc[10]
    

0                       TLE,Sep 24 2015 13:33:46.359360,1  
1                                                   40912U 
2                                                   15049P 
3                                           15267.56511990 
4                                                .00004188 
5                                                  00000-0 
6                                                  23243-3 
7                                                         0
8                                                          
9                                                       1,2
10                                                 40912   
11                                                 97.4626 
12                                                275.6379 
13                                                 0016731 
14                                               232.1752  
15                                                127.8620 
16                                      

In [116]:
tle_df.iloc[10000]

0                       TLE,May 16 2023 13:49:08.643072,1  
1                                                   40912U 
2                                                   15049P 
3                                           23136.57579448 
4                                                .00041294 
5                                                  00000-0 
6                                                  97537-3 
7                                                         0
8                                                          
9                                                       1,2
10                                                 40912   
11                                                 97.4193 
12                                                153.7840 
13                                                 0009668 
14                                               179.8024  
15                                                300.4152 
16                                      

In [117]:
for i in range(len(tle_df)):
    tle_df[i] = tle_df[i].split(' ')

AttributeError: 'Series' object has no attribute 'split'

In [688]:
len(tle_df)

10371

In [689]:
tle_df

0        [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903...
1        [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903...
2        [TLE,Sep, 20, 2015, 08:36:33.911712,1, , 40903...
3        [TLE,Sep, 20, 2015, 11:55:28.663392,1, , 40903...
4        [TLE,Sep, 20, 2015, 16:23:49.615296,1, , 40903...
                               ...                        
10366    [TLE,Apr, 20, 2023, 17:13:32.215008,1, , 40903...
10367    [TLE,Apr, 20, 2023, 17:13:32.215008,1, , 40903...
10368    [TLE,Apr, 21, 2023, 09:18:18.054144,1, , 40903...
10369    [TLE,Apr, 22, 2023, 10:28:18.609600,1, , 40903...
10370    [TLE,Apr, 22, 2023, 10:28:18.609600,1, , 40903...
Name: Output, Length: 10371, dtype: object

In [690]:
tle_df_try

,"TLE,Sep",20,2015,"06:58:37.763904,1",,40903U,15049E,15263.29071486,-.00000064,00000-0,...,271.4568,0012305,264.7276,219.0254,15.12843012,52,None,None,None,


In [691]:
for h in range(len(tle_df)):
    for i in len(tle_df[h]:
        if i == 'None' or i == '':
            del i
        else: tle_df_try.iloc[h][i] = i

SyntaxError: invalid syntax (2128493293.py, line 2)

In [569]:
tle_df_try

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
1,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
2,68,68,68,68,68,68,68,68,68,68,...,68,68,68,68,68,68,68,68,68,68
3,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
4,116,116,116,116,116,116,116,116,116,116,...,116,116,116,116,116,116,116,116,116,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10366,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,...,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369
10367,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,...,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369
10368,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,...,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470
10369,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,...,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647


In [561]:
tle_df[0]

['TLE,Sep',
 '20',
 '2015',
 '06:58:37.763904,1',
 '',
 '40903U',
 '15049E',
 '15263.29071486',
 '-.00000064',
 '00000-0',
 '00000+0',
 '09999',
 '240903',
 '97.4551',
 '271.4568',
 '0012305',
 '264.7276',
 '219.0254',
 '15.12843012',
 '52',
 'None',
 '']

In [544]:
tle_df[0][20]

'None'

In [536]:
tle_df_try = pd.DataFrame(columns=range(30),index=range(len(tle_df)))

In [537]:
for i in range(len(tle_df_try)):
    tle_df_try.iloc[i] = tle_df.iloc[i]

tle_df_try

ValueError: could not broadcast input array from shape (24,) into shape (30,)

In [505]:
for i in range(len(tle_df_try)):
    for h in range(22):
        tle_df_try[i][h] = tle_df[i][h]

KeyError: 22

TypeError: unhashable type: 'list'

Empty DataFrame
Columns: [[TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903U, 15049E, 15263.29071486, -.00000064, 00000-0, 00000+0, 09999, 240903, 97.4551, 271.4568, 0012305, 264.7276, 219.0254, 15.12843012, 52, None, None, None, ], [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903U, 15049E, 15263.29071486, -.00000064, 00000-0, 00000+0, 09999, 240903, 97.4551, 271.4568, 0012305, 264.7276, 219.0254, 15.12843012, 52, None, None, None, ], [TLE,Sep, 20, 2015, 08:36:33.911712,1, , 40903U, 15049E, 15263.35872583, -.00000064, 00000-0, 00000+0, 09993, 240903, 97.4551, 271.5232, 0012305, 264.4928, 229.4290, 15.12843012, 68, None, None, None, ], [TLE,Sep, 20, 2015, 11:55:28.663392,1, , 40903U, 15049E, 15263.49685953, -.00000064, 00000-0, 00000+0, 09991, 240903, 97.4594, 271.6596, 0011878, 263.4082, 262.3431, 15.12835531, 85, None, None, None, ], [TLE,Sep, 20, 2015, 16:23:49.615296,1, , 40903U, 15049E, 15263.68321314, -.00000064, 00000-0, 00000+0, 09990, 240903, 97.4579, 271.8457, 0011878, 262.7611, 19